In [ ]:
#REGULAR NEURONS
#14 neurons, connected to Poisson spikegen
name_of_run= "sigmadelta"

In [ ]:
#!pip install scipy

In [ ]:
import samna
import samna.dynapse1 as dyn1
import time

import sys
# change the path to '/home/class_NI2021/ctxctl_contrib' on zemo
sys.path.insert(1, '/home/class_NI2021/ctxctl_contrib')
from Dynapse1Constants import *
import Dynapse1Utils as ut
import NetworkGenerator as n
from NetworkGenerator import Neuron
import numpy as np
from functions import find_max_time,find_min_time,isi_calc,interspike

import matplotlib
# Display plots inline, directly below the cell that produced it.
%matplotlib inline 
import matplotlib.pyplot as plt


from scipy import signal

from numba import njit 
from scipy.interpolate import interp1d


# open DYNAP-SE1 board to get Dynapse1Model
device_name = "my_dynapse1"
# change the port numbers to not have conflicts with other groups
store = ut.open_dynapse1(device_name, gui=False, sender_port=119325, receiver_port=632943)
model = getattr(store, device_name)

np.random.seed(77)


In [ ]:
# Ignore warnings  
#import warnings
#warnings.filterwarnings("ignore",category=UserWarning)

def gen_param_group_1core():
    paramGroup = dyn1.Dynapse1ParameterGroup()
    # THR, gain factor of neurons
    paramGroup.param_map["IF_THR_N"].coarse_value = 5 #5
    paramGroup.param_map["IF_THR_N"].fine_value = 80

    # refactory period of neurons
    paramGroup.param_map["IF_RFR_N"].coarse_value = 4
    paramGroup.param_map["IF_RFR_N"].fine_value = 128

    # leakage of neurons
    paramGroup.param_map["IF_TAU1_N"].coarse_value = 4
    paramGroup.param_map["IF_TAU1_N"].fine_value = 80

    # turn off tau2
    paramGroup.param_map["IF_TAU2_N"].coarse_value = 7
    paramGroup.param_map["IF_TAU2_N"].fine_value = 255

    # turn off DC
    paramGroup.param_map["IF_DC_P"].coarse_value = 0
    paramGroup.param_map["IF_DC_P"].fine_value = 0

    # leakage of AMPA
    paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
    paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 80

    # gain of AMPA
    paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 4
    paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 80

    # weight of AMPA
    # NOTE: !!!!!!!!!!!!!!!!!!!!!!! remember to set the weight of AMPA !!!!!!!!!!!!!!!!!!!!!!!
    paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
    paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 80 

    # leakage of NMDA
    paramGroup.param_map["NPDPIE_TAU_S_P"].coarse_value = 4
    paramGroup.param_map["NPDPIE_TAU_S_P"].fine_value = 80

    # gain of NMDA
    paramGroup.param_map["NPDPIE_THR_S_P"].coarse_value = 4
    paramGroup.param_map["NPDPIE_THR_S_P"].fine_value = 80

    # weight of NMDA
    paramGroup.param_map["PS_WEIGHT_EXC_S_N"].coarse_value = 0
    paramGroup.param_map["PS_WEIGHT_EXC_S_N"].fine_value = 0

    # leakage of GABA_A (shunting)
    paramGroup.param_map["NPDPII_TAU_F_P"].coarse_value = 4
    paramGroup.param_map["NPDPII_TAU_F_P"].fine_value = 80

    # gain of GABA_A (shunting)
    paramGroup.param_map["NPDPII_THR_F_P"].coarse_value = 4
    paramGroup.param_map["NPDPII_THR_F_P"].fine_value = 80

    # weight of GABA_A (shunting)
    paramGroup.param_map["PS_WEIGHT_INH_F_N"].coarse_value = 0
    paramGroup.param_map["PS_WEIGHT_INH_F_N"].fine_value = 0

    # leakage of GABA_B
    paramGroup.param_map["NPDPII_TAU_S_P"].coarse_value = 2
    paramGroup.param_map["NPDPII_TAU_S_P"].fine_value = 80

    # gain of GABA_B
    paramGroup.param_map["NPDPII_THR_S_P"].coarse_value = 4
    paramGroup.param_map["NPDPII_THR_S_P"].fine_value = 80

    # weight of GABA_B
    paramGroup.param_map["PS_WEIGHT_INH_S_N"].coarse_value = 4
    paramGroup.param_map["PS_WEIGHT_INH_S_N"].fine_value = 80

    # other advanced parameters
    paramGroup.param_map["IF_NMDA_N"].coarse_value = 0
    paramGroup.param_map["IF_NMDA_N"].fine_value = 0

    #adaptation parameter 1/4
    paramGroup.param_map["IF_AHTAU_N"].coarse_value = 0
    paramGroup.param_map["IF_AHTAU_N"].fine_value = 0

    #adaptation parameter 2/4
    paramGroup.param_map["IF_AHTHR_N"].coarse_value = 0 #0
    paramGroup.param_map["IF_AHTHR_N"].fine_value = 0 #0

    #adaptation parameter 3/4
    paramGroup.param_map["IF_AHW_P"].coarse_value = 0 #0
    paramGroup.param_map["IF_AHW_P"].fine_value = 0 #0

    #adaptation parameter 4/4
    paramGroup.param_map["IF_CASC_N"].coarse_value = 0 #0
    paramGroup.param_map["IF_CASC_N"].fine_value = 0 #0

    paramGroup.param_map["PULSE_PWLK_P"].coarse_value = 4
    paramGroup.param_map["PULSE_PWLK_P"].fine_value = 106

    paramGroup.param_map["R2R_P"].coarse_value = 3
    paramGroup.param_map["R2R_P"].fine_value = 85

    paramGroup.param_map["IF_BUF_P"].coarse_value = 3
    paramGroup.param_map["IF_BUF_P"].fine_value = 80

    return paramGroup

In [ ]:


# set initial (proper) parameters of 16 cores
paramGroup = gen_param_group_1core()
for chip in range(4):
    for core in range(4):
        model.update_parameter_group(paramGroup, chip, core)

In [ ]:
#CREATE NETWORK
net_gen = n.NetworkGenerator()
#net_gen.clear_network()


spikegen_up = Neuron(0,0,1,True)
spikegen_down = Neuron(0,0,2,True)

# select the neurons
chip_r = 0
core_r = 0
num_neurons_r = 14
start_neuron_id_r = 16
neuron_ids_r = []
neurons_r = []

for id in range(num_neurons_r):
    neuron_ids_r.append((chip_r,core_r,start_neuron_id_r + id))
print("Neuron for regular indexes:{}".format(neuron_ids_r))

for neuron_id in neuron_ids_r:
    neurons_r.append(Neuron(neuron_id[0],neuron_id[1],neuron_id[2],False))


#create synapses spikegen-neurons 
for i in range(14):
    net_gen.add_connection(spikegen_up, neurons_r[i], dyn1.Dynapse1SynType.AMPA)


for i in range(14):
    net_gen.add_connection(spikegen_down, neurons_r[i], dyn1.Dynapse1SynType.GABA_B)
#default was AMPA, should we change to NMDA? 


# print the network so you can double check (optional)
net_gen.print_network()
# make a dynapse1config using the network
new_config = net_gen.make_dynapse1_configuration()
# apply the configuration
model.apply_configuration(new_config)



In [ ]:
from scipy import signal
#Firstly we define the head direciton signal.

#Following the paper of Sadeghi et al., we construct a broadband noise stimuli characterized by a Gaussian 
#distribution of rotational velocities with mean = 0 and std = 20 degrees/s. The Signal is then low-pass filtered at 
#30 Hz.

def gaussian_head_signal(mu, sigma, duration, freq):
    """
    Generates a time series signal of a broadband noise signal characterized by a Gaussian distribution with a 
    certain mean and standard deviation, corresponding to angular velocity. Plots the signal in time
    
    Args: 
        mu (float): mean of the gaussian distribution
        sigma (float): standard deviaiton of gaussian distribution
        duration (np.ndarray): number of time points to sample (s)
        freq (float): frequency boundary for low-pass filter
        
    Returns:
            head_signal (np.ndarray): signal of head direction velocities generated by the gaussian
            head_signal_filtered (np.ndarray): lowpass filtered head velocity signal
            freq_signal_filtered (np.ndarray): lowpass filtered frequency signal
        
    """
    
    
    
    x = np.linspace(0, duration, int(duration*1e3)) 
    head_signal = np.random.normal(mu, sigma*(np.pi/180), int(duration*1e3))#np.random.normal(mu, sigma*(np.pi/180), int(duration*1e3))
    
    
    sos = signal.butter(10, 30, 'lowpass', fs=1000, output='sos')
    freq_signal_filtered = signal.sosfilt(sos, head_signal/(2*np.pi))
    
    
    head_signal_filtered = freq_signal_filtered*(2*np.pi)*(180/np.pi)
    
    plt.figure(figsize=(8, 13))
    
    plt.subplot(211)
    plt.plot(x, head_signal, color='tab:blue')
    plt.title("Unfiltered head velocity stimulus")
    plt.xlabel("time (s)")
    plt.ylabel("rad/s")

    plt.subplot(212)
    plt.plot(x, head_signal_filtered, color='tab:orange')
    plt.title("Filtered head velocity stimulus")
    plt.xlabel("time (s)")
    plt.ylabel("degrees/s")
    
    #plt.subplot(224)
    #plt.plot(x, freq_signal_filtered, color='tab:red')
    
    plt.show()
    
    return head_signal, head_signal_filtered, freq_signal_filtered







In [ ]:
#Sigma delta modulator for the head direction signal
from numba import njit 
from scipy.interpolate import interp1d

#Usage:
    #converts signal into UP and DOWN spikes using the Adaptive Delta Modulation scheme.
    #spike times are determined through linear interpolation between the current and the next sampling point
    
#Input Parameters:
    #input_signal (array): input signal
    #threshold_UP (float): threshold for UP spikes to occur
    #threshold_DOWN (float): threshold for DOWN spikes to occur
    #sampling_frequency (float): sampling frequency of the input signal (1/dt)
    #refractory_period_duration (float): refractory period in which spikes can not occur (NEEDS TO BE AT LEAST 1/sampling_frequency)
    #return_indices (bool): if False, returns only spike times. If True, retuns a new time array along with UP and DOWN indices arrays which indicate when respective spikes occured
    #index_dt (float): dt of ouput indices (for higher accuracy, use non-numba ADM as it will be faster)
    #Vdc_reset (bool): determines whether signal continues to integrate in the refractory period (False)
    #or whether the integration is reset and signal needs to integrate the full threshold after the refractory period (True)
    
#Output Parameters:
    #spike_t_up (array): list of precise UP spike times
    #spike_t_dn (array): list of precise DOWN spike times
    #times_interpolated (array): time array for spike indices arrays (dt = index_dt)
    #spike_idx_up (array of shape times_interpolated): closest times to UP spikes in times_interpolated are set to 1; rest is 0 
    #spike_idx_dn (array of shape times_interpolated): closest times to DOWN spikes in times_interpolated are set to 1; rest is 0 
    
def ADM(input_signal,threshold_UP,threshold_DOWN,sampling_frequency,refractory_period_duration,return_indices = True,index_dt = 1e-4, Vdc_reset = True):
    dt = 1/sampling_frequency
    end_time = len(input_signal)*dt
    times = np.linspace(0,end_time,len(input_signal))
    if refractory_period_duration < dt:
        interpolation_factor = 1
        while dt > refractory_period_duration:
            interpolation_factor += 1
            dt = 1/(sampling_frequency*interpolation_factor)
        f = interp1d(times, input_signal)
        times = np.arange(0,times[-1],dt)
        input_signal = f(times)
        sampling_frequency = 1/times[1]
    spike_t_up,spike_t_dn,times_interpolated,spike_idx_up,spike_idx_dn = ADM_numba(input_signal,threshold_UP,threshold_DOWN,sampling_frequency,refractory_period_duration,return_indices,index_dt, Vdc_reset)
    return (spike_t_up,spike_t_dn,times_interpolated,spike_idx_up,spike_idx_dn)

@njit(fastmath=True, parallel = True)
def ADM_numba(input_signal,threshold_UP,threshold_DOWN,sampling_frequency,refractory_period_duration,return_indices = True,index_dt = 1e-4, Vdc_reset = True):

    dt = 1/sampling_frequency
    end_time = len(input_signal)*dt
    times = np.linspace(0,end_time,len(input_signal)).astype(np.float64)
    DC_Voltage = input_signal[0]
    remainder_of_refractory = 0
    spike_t_up = times[0:2]
    spike_t_dn = times[0:2]
    interpolate_from = 0.0
    interpolation_activation = 0
    intercept_point=0
    for i,t in enumerate(times):
        slope = ((input_signal[i]-input_signal[i-1])/dt)
        if remainder_of_refractory > dt:
            remainder_of_refractory = remainder_of_refractory-dt
            interpolation_activation = 1
            
            if Vdc_reset:
                DC_Voltage = (input_signal[i] - interpolate_from*slope)

        else:
            
            if interpolation_activation == 1:
                interpolate_from = interpolate_from-remainder_of_refractory
                Vbelow = (input_signal[i] - interpolate_from*slope)
                DC_Voltage = Vbelow
                
                if DC_Voltage + threshold_UP < (input_signal[i] - interpolate_from*slope):
                    spike_t_up = np.append(spike_t_up,t-interpolate_from)
                    remainder_of_refractory = refractory_period_duration 
                    DC_Voltage = (input_signal[i] - interpolate_from*slope)
                    interpolation_activation = 0
                    continue

                if DC_Voltage - threshold_DOWN > (input_signal[i] - interpolate_from*slope):
                    spike_t_dn= np.append(spike_t_dn,t-interpolate_from)
                    remainder_of_refractory = refractory_period_duration 
                    DC_Voltage = (input_signal[i] - interpolate_from*slope)
                    interpolation_activation = 0
                    continue
                
            else:
                Vbelow = input_signal[i-1]
                interpolate_from = dt

            if DC_Voltage + threshold_UP <= input_signal[i]:
                intercept_point = t-interpolate_from+((threshold_UP+DC_Voltage-Vbelow)/slope)
                spike_t_up = np.append(spike_t_up,intercept_point)
                interpolate_from = t+dt-intercept_point
                remainder_of_refractory = refractory_period_duration 
                DC_Voltage = (input_signal[i] - interpolate_from*slope)

            elif DC_Voltage - threshold_DOWN >= input_signal[i]:
                intercept_point = t-interpolate_from+((-threshold_DOWN+DC_Voltage-Vbelow)/slope)
                spike_t_dn = np.append(spike_t_dn,intercept_point)
                interpolate_from = t+dt-intercept_point
                remainder_of_refractory = refractory_period_duration 
                DC_Voltage = (input_signal[i] - interpolate_from*slope)

            interpolation_activation = 0
                        
    index =[0,1]
    spike_t_up = np.delete(spike_t_up, index)
    spike_t_dn = np.delete(spike_t_dn, index)
    if return_indices:
        times_interpolated =   np.arange(0,end_time,index_dt)
        spike_idx_up = np.zeros_like(times_interpolated)
        spike_idx_dn = np.zeros_like(times_interpolated)
        idxdn = np.searchsorted(times_interpolated,spike_t_dn)
        spike_idx_dn[idxdn] = 1
        idxup = np.searchsorted(times_interpolated,spike_t_up)
        spike_idx_up[idxup] = 1
    return (spike_t_up,spike_t_dn,times_interpolated,spike_idx_up,spike_idx_dn)

In [ ]:
# Now test on mu = 0 and std = 20 deg as stated in the paper
mu = 0
sigma = 20
duration = 1
freq = 30

np.random.seed(77)
head_signal, head_signal_filtered, freq_signal  = gaussian_head_signal(mu, sigma, duration, freq)

In [ ]:
input_signal = head_signal_filtered*0.01
threshold_UP = 0.03
threshold_DOWN = 0.03
dt = 1e-3
sampling_frequency = 1/dt
refractory_period_duration = 1/sampling_frequency

spike_t_up, spike_t_down, times_interpolated,spike_idx_up, spike_idx_down =  ADM(input_signal,threshold_UP,threshold_DOWN,
                                                  sampling_frequency,refractory_period_duration,
                                                  return_indices = True,index_dt = 1e-3, Vdc_reset = True)

plt.figure(figsize = (15,10))
plt.plot(head_signal_filtered)
plt.vlines(spike_t_down*1000, -1, 1, colors = "g",label = "up spikes")
plt.vlines(spike_t_up*1000, -1, 1, colors = "r",label="down spikes")
plt.legend()
#print(spike_t_down)
#print(spike_t_down)

In [ ]:
# get the fpga spike gen from Dynapse1Model
#spike_t_up = spike_t_up.tolist()
#print(spike_t_up.dtype)
#s = np.array([0.10110785, 0.11072627 ,0.1183185 , 0.12491149, 0.13090515 ,0.136508])
#s1 = spike_t_up[0:20]
#s2 = np.linspace(0,5,320)

#SPIKES UP
fpga_spike_gen_up = model.get_fpga_spike_gen()

spikegen_id = 1
indices = [spikegen_id]*len(spike_t_up)

post_chip = chip_r
target_chips = [post_chip]*len(spike_t_up)

# set up the fpga_spike_gen
ut.set_fpga_spike_gen(fpga_spike_gen_up, spike_t_up , indices = indices, target_chips=target_chips, isi_base= 900, repeat_mode = False)

#SPIKES DOWN
fpga_spike_gen_down = model.get_fpga_spike_gen()

spikegen_id = 2
indices = [spikegen_id]*len(spike_t_down)

post_chip = chip_r
target_chips = [post_chip]*len(spike_t_down)

# set up the fpga_spike_gen
ut.set_fpga_spike_gen(fpga_spike_gen_down, spike_t_down , indices = indices, target_chips=target_chips, isi_base= 900, repeat_mode = False)



#INDICES SET TO ONE OF THE PFGA, NEED TO CONNECT ALL TO ONE

# remember to start the spikegen


In [ ]:
#monitor neurons
monitored_global_nids_r = ut.get_global_id_list(neuron_ids_r)

#monitored_global_nids_r = [1]

# create a graph to monitor the spikes of these neurons
graph, filter_node, sink_node = ut.create_neuron_select_graph(model, monitored_global_nids_r)

#1 second 
duration = 1

# start monitor
graph.start()

# start the stimulus
fpga_spike_gen_up.start()
fpga_spike_gen_down.start()
print("Starting Poisson spikegen for regular neurons")
#fpga_spike_gen.start()

# ------------ get events -----------
# clear the output buffer
sink_node.get_buf()

# sleep
time.sleep(duration)

# get the events accumulated during the past 2 sec
events = sink_node.get_buf()

# ------------ get events -----------

# stop the stimulus
fpga_spike_gen_up.stop()
fpga_spike_gen_down.stop()
print("Stopping Poisson spikegen for regular neurons")

# stop graph
graph.stop()


#timestamps of spikes
timestamps = []
frequencies = []

for i in range(num_neurons_r):
    neur_timestamps_id = i
    timestamps_one_neuron = []
    for evt in events:
        if (evt.neuron_id-start_neuron_id_r) == neur_timestamps_id:
            timestamps_one_neuron.append(evt.timestamp)

    frequency_one_neuron = len(timestamps_one_neuron)/duration
    print("Neuron {} timestamps: {}".format(i,timestamps_one_neuron))
    print("Spiking frequecy: {} Hz".format(frequency_one_neuron))

    timestamps.append(timestamps_one_neuron)
    frequencies.append(frequency_one_neuron)
'''
#monitor spikegen
spikegen_timestamps = []
for evt in events:
    spikegen_timestamps.append(evt.timestamp)

print("spikegen timestamps:  {}".format(spikegen_timestamps))
'''


In [ ]:
timestamps_original = np.copy(timestamps)

In [ ]:
min_time = find_min_time(timestamps_original)
timestamps_preprocess = []
timestamps_ms = []

for timestamp in timestamps: 
    timestamps_preprocess.append(np.subtract(timestamp,min_time))
    timestamps_ms.append(np.subtract(timestamp,min_time)/1000)

In [ ]:

fig, axs = plt.subplots(7,2, sharex=True,figsize=(15,50),facecolor=(1, 1, 1))  
for i, ax in enumerate(axs.ravel()): 
    ax.set_title("Regular neuron #{}".format(i)) 
    ax.hist(isi_calc(timestamps_ms[i]))
    ax.xaxis.set_tick_params(labelbottom=True)
    ax.set_xlabel("[ms]")
#plt.savefig("plots_regular/SDHistogram_{}.png".format(name_of_run))


plt.figure(figsize=(20,10),facecolor=(1, 1, 1))
plt.ylabel("Neuron index")
plt.title("Raster plot of regular neurons")
_ = plt.eventplot(timestamps_ms,linelengths=0.5)
#plt.savefig("plots_regular/SDRaster_{}.png".format(name_of_run))


plt.figure(facecolor=(1, 1, 1))
plt.plot(frequencies)
plt.title("Frequencies of firing")
plt.xlabel("Neuron nr.")
plt.ylabel("Frequency [Hz]")
plt.ylim(0,100)
#plt.savefig("plots_regular/SDFrequencies_{}.png".format(name_of_run))

In [ ]:
# close Dynapse1
#ut.close_dynapse1(store, device_name)

In [ ]:
def binary_signal(signal, recording_duration):
    """
    Given a spike train with the spike times, create a binary signal of spikes
    
    Args:
        signal (np.ndarray): array of spike times
        recording_duration (float): duration of time the recording has lasted in ms
        
    Returns: 
            bsignal (np.ndarray): time series of binary signal
    """
    bsignal = np.zeros((recording_duration,))
    bsignal[signal] = 1
    
    return bsignal

In [ ]:
#we recorded the signal for 1000 micro s
recording_duration = 1000

#as a trial we will use the 0th neuron timestamp

spike_response = timestamps_ms[0]

#Now we wish to plot the spiking response inverted and non-inverted.

response = binary_signal(np.round(spike_response).astype(int),recording_duration)
response_flipped = binary_signal(np.round(-spike_response).astype(int),recording_duration)

plt.figure()
plt.plot(response)
plt.title("Binary spiking signal")

plt.figure()
plt.plot(response_flipped)
plt.title("Binary inverted spiking signal")

plt.figure()
stimuli = binary_signal((spikes).astype(int), recording_duration)
plt.plot(stimuli)
plt.title("Input spikes")




In [ ]:
def reconstruct_signal(response, response_flipped, stimuli):
    """
    Reconstructs the signal as shown in the neural variability paper, by convolving the spike train resposne with the
    kernel that minimizes the MSE.
    
    Args:
        response(np.ndarray): binary array contining the spikes
        response_flipped (np.ndarray): the flipped spike times (multiplied by -1)
        stimuli (np.ndarray): initial binary spike times
    
    Returns:
            signal_estimate(np.ndarray): vaues of the reconstructed signal esitmate
    """

    #Computes the stimulus-response cross spectrum (the functio takes the fourier transform of the signal)
    f_rs, Prs = signal.csd(response_flipped,stimuli,scaling = "spectrum", nperseg = 1000)
    plt.semilogy(f_rs, Prs)
    plt.plot()
    plt.title("Stimulus-response cross spectrum")

    
    #Computes the power spectrum of the response: note that when using the welch function, very different results are
    #observed. Uncomment it and comment out the periodgram to see.
    
    f_rr, Prr = signal.welch(response, nperseg=1000)  
    #f_rr, Prr = signal.periodogram(response,scaling = "spectrum")
    plt.figure()
    plt.semilogy(f_rr, Prr)
    plt.title("Output response power-spectrum")

    #Kernel in the fourier domain that minimizes the MSE
    K_fourier = Prs / Prr
    
    K_fourier_real = K_fourier.real
    
    plt.figure()
    plt.plot(K_fourier)
    plt.title("Fourier transform of the kernel")
    
    #Inverting the fourier transform to get the kernel for the convolution
    kernel = np.fft.ifft(K_fourier_real)
    kernel_real = np.real(kernel)
    
    plt.figure()
    plt.plot(kernel)
    plt.plot(kernel_real)
    plt.title("Reconstruction kernel")

    #signal estimate by convolving the spike responses with the kernal
    signal_estimate = np.convolve(response, kernel_real)

    plt.figure()
    plt.plot(signal_estimate)
    plt.title("Signal reconstruction")
    
    return signal_estimate, K_fourier_real, kernel, kernel_real





In [ ]:
#signal_reconstructed, K_fourier, kernel, kernel_real = reconstruct_signal(response, response_flipped, stimuli)

In [ ]:
import scipy 
gauss = scipy.signal.gaussian(1000, 8, sym=True)
plt.plot(gauss)
plt.ylabel("Amplitude")
plt.xlabel("Time [ms]")

print(gauss.shape)
recon = np.convolve(gauss,response)
plt.figure()
plt.plot(recon[0:1501])
plt.xlabel("Time [ms]")
plt.ylabel("Amplitude")
#plt.figure()
#plt.plot(response)

plt.figure()
plt.plot(head_signal_filtered)
plt.xlabel("Time [ms]")
plt.ylabel("Amplitude (deg/s)")



In [ ]:
#regular correlation for neuron 0

from scipy.stats import pearsonr

corr, _ = pearsonr(recon[500:1500], head_signal_filtered)
corr

In [ ]:
def normalize(signal, a, b):
      # solving system of linear equations one can find the coefficients
      A = np.min(signal)
      B = np.max(signal)
      C = (a-b)/(A-B)
      k = (C*A - a)/C
      return (signal-k)*C

In [ ]:
#regular neuron normalized correlation with shift

h_n = normalize(head_signal_filtered, -1, 1)
r_n = normalize(recon[480:1480], -1, 1)

corr_n, _ = pearsonr(r_n, h_n)
print(corr_n)

plt.plot(h_n, label = "head velocity stimulus")
plt.plot(r_n, label = "stimulus reconstruction")
plt.title("Normalized plots of stimulus and output")
plt.xlabel("Time (ms)")
plt.ylabel("Normalized head velocity amplitude")
plt.legend(loc = "upper right")

In [ ]:
def coherence(stimulus, output):
    """
    Calculates the coherence of the stimulus and the output of the neurons
    
    Args:
        stimulus (np.ndarray): time series stimulus
        output (np.ndarray): time series output of neuron
        
    Return:
            coherence (np.ndarray): value of coherence of output and stimulus signal for different frequency values
    """
    #Calculate the stimulus-response cross spectrum
    frs, Prs = signal.csd(stimulus, output, nperseg = 1000)
    
    #Power spectrum of the spike train (output)
    #frr, Prr = signal.welch(output)
    
    frr, Prr = signal.periodogram(output)
    
    #Power spectrum of the stimulus
    #fss, Pss = signal.welch(stimulus)
    
    fss, Pss = signal.periodogram(stimulus)
    
    Cf = (np.abs(Prs)**2)/(Prr*Pss) #coherence values
    
    
    
    return Cf, frs, frr, fss, Prs, Pss, Prr

In [ ]:
Cf, frs, frr, fss, Prs, Pss, Prr= coherence(h_n, r_n)
print("Mean coherence of stimulus and output is", np.mean(Cf))

In [ ]:
#spike sorting by up and down
import numpy as np
spikes = np.arange(7)
up_spikes = spikes[0:2]
down_spikes = spikes[2:]

#up spikes have index of 1
ones = np.ones(up_spikes.shape)

#down spikes have index 2
twos = 2*np.ones(down_spikes.shape)

indx_concatenate = np.concatenate((ones, twos))

spikes_concatenate = np.concatenate((up_spikes, down_spikes))

sorted_idx_spike = np.argsort(spikes_concatenate)

sorted_idx_spikegen = indx_concatenate[sorted_idx_spike]



In [ ]:
print(sorted_idx_spike)
print(sorted_idx_spikegen)

In [ ]:
pos = np.array([1, 2])
neg = np.array([-2, -1])

from scipy.stats import pearsonr

corr, _ = pearsonr(pos, neg)

